# Generate Examples for Entity Relationship Extraction

- Taking datasets from Huggingface containing news articles and generate entities and relationships out of each news article.
- Save them as DSPy examples locally to be used for fine-tuning prompt instructions.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import dspy
import asyncio
import os
import numpy as np
from dotenv import load_dotenv
from datasets import load_dataset
import logging
import pickle

from nano_graphrag._utils import compute_mdhash_id
from nano_graphrag.entity_extraction.extract import generate_dataset

/opt/homebrew/Caskroom/miniconda/base/envs/nano-graphrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
WORKING_DIR = "./nano_graphrag_cache_generate_dspy_examples"

load_dotenv()

logging.basicConfig(level=logging.WARNING)
logging.getLogger("nano-graphrag").setLevel(logging.DEBUG)

np.random.seed(1337)

In [4]:
system_prompt = """
    You are a world-class AI system, capable of complex reasoning and reflection. 
    Reason through the query, and then provide your final response. 
    If you detect that you made a mistake in your reasoning at any point, correct yourself.
    Think carefully.
"""
lm = dspy.OpenAI(
    model="deepseek-chat", 
    model_type="chat", 
    api_key=os.environ["DEEPSEEK_API_KEY"], 
    base_url=os.environ["DEEPSEEK_BASE_URL"], 
    system_prompt=system_prompt, 
    temperature=1.0,
    top_p=1.0,
    max_tokens=4096
)
dspy.settings.configure(lm=lm, experimental=True)

In [5]:
train_len = 100
val_len = 100
dev_len = 200

In [6]:
os.makedirs(WORKING_DIR, exist_ok=True)
entity_relationship_trainset_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news_trainset.pkl")
entity_relationship_valset_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news_valset.pkl")
entity_relationship_devset_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news_devset.pkl")

In [7]:
fin_news = load_dataset("ashraq/financial-news-articles")
cnn_news = load_dataset("AyoubChLin/CNN_News_Articles_2011-2022")
fin_shuffled_indices = np.random.permutation(len(fin_news['train']))
cnn_train_shuffled_indices = np.random.permutation(len(cnn_news['train']))
cnn_test_shuffled_indices = np.random.permutation(len(cnn_news['test']))
train_data = cnn_news['train'].select(cnn_train_shuffled_indices[:train_len])
val_data = cnn_news['test'].select(cnn_test_shuffled_indices[:val_len])
dev_data = fin_news['train'].select(fin_shuffled_indices[:dev_len])

/opt/homebrew/Caskroom/miniconda/base/envs/nano-graphrag/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [8]:
train_chunks = {compute_mdhash_id(text, prefix=f"chunk-trainset-"): {"content": text} for text in train_data["text"]}
trainset = asyncio.run(generate_dataset(chunks=train_chunks, filepath=entity_relationship_trainset_path))

INFO:nano-graphrag:Saved 100 examples with keys: ['input_text', 'entities', 'relationships'], filtered 0 examples


In [9]:
trainset[:5]

[Example({'input_text': ' (CNN)Former Russian spy Sergei Skripal and his daughter, Yulia, are continuing to recover from a nerve agent attack, and their rehabilitation has been "slow and extremely painful," the daughter said Wednesday. Sergei, 66, and Yulia Skripal were found March 4 slumped on a bench in Salisbury, England, after being exposed to novichok, a military-grade nerve agent.While Yulia Skripal, 33, spent 20 days in a coma before she was released from the hospital in April and taken to a safe location, her father remained hospitalized until last week. "We are so lucky to have both survived this attempted assassination," she said Wednesday from an undisclosed location, her first public appearance since the attack. "I don\'t want to describe the details, but the clinical treatment was invasive, painful and depressing. Our recovery has been slow and extremely painful."JUST WATCHEDYulia Skripal released from hospitalReplayMore Videos ...MUST WATCHYulia Skripal released from hosp

In [10]:
val_chunks = {compute_mdhash_id(text, prefix=f"chunk-valset-"): {"content": text} for text in val_data["text"]}
valset = asyncio.run(generate_dataset(chunks=val_chunks, filepath=entity_relationship_valset_path))

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 400 Bad Request"
ERROR:nano-graphrag:Error in TypedEntityRelationshipExtractor: Error code: 400 - {'error': {'message': 'Content Exists Risk', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}


INFO:nano-graphrag:Saved 99 examples with keys: ['input_text', 'entities', 'relationships'], filtered 1 examples


In [11]:
valset[:5]

[Example({'input_text': 'Story highlightsPortugal qualifies for Euro 2016 finalDefeats Wales 2-0 in semifinalCristiano Ronaldo and Nani on targetFrance or Germany awaits in the final (CNN)Cristiano Ronaldo proved himself the man for the big occasion once again by firing Portugal into the final of Euro 2016 Wednesday.Ronaldo\'s second-half header and a strike by his former Manchester United teammate Nani secured a 2-0 win against Wales in Lyon.Follow @cnnsport\n\nPortugal, which last reached the final of the competition in 2004, will face either host nation France or world champion Germany in Paris on Sunday.This contest had been billed as clash between two of the world\'s top players -- Ronaldo and his Real Madrid teammate Gareth Bale.But Bale, who has enjoyed a stellar tournament for a country which had never before qualified for the tournament, failed to match the exploits of his rival.Read MoreRonaldo, who equaled Michel Platini\'s record of scoring nine career goals at the Euros, w

In [12]:
dev_chunks = {compute_mdhash_id(text, prefix=f"chunk-devset-"): {"content": text} for text in dev_data["text"]}
devset = asyncio.run(generate_dataset(chunks=dev_chunks, filepath=entity_relationship_devset_path))

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


INFO:nano-graphrag:Saved 197 examples with keys: ['input_text', 'entities', 'relationships'], filtered 3 examples


In [13]:
devset[:5]

[Example({'input_text': "As students from Marjory Stoneman Douglas High School confront lawmakers with demands to restrict sales of assault rifles, there were warnings by the president of the Florida school administration association that schools in the state were vulnerable to such an attack.\nHighlights\n“Nikolas Cruz was able to purchase an assault rifle before he was able to buy a beer,” said Stoneman Douglas student Laurenzo Prado, referring to a Florida law that allows people as young as 18 to buy assault weapons. Students galvanized by the deadly mass shooting at the Florida high school confronted lawmakers with demands to restrict sales of assault rifles, while President Donald Trump suggested arming teachers as a way to stop more U.S. rampages.\nTwo weeks before a gunman fatally shot 17 people at a Florida high school, Bill Lee, the president of the state’s school administrators association, warned that Florida’s schools were vulnerable to just such an attack. “It’s not a matt

In [14]:
loaded_trainset = pickle.load(open(entity_relationship_trainset_path, "rb"))
loaded_valset = pickle.load(open(entity_relationship_valset_path, "rb"))
loaded_devset = pickle.load(open(entity_relationship_devset_path, "rb"))
assert loaded_trainset == trainset
assert loaded_valset == valset
assert loaded_devset == devset